In [ ]:

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader
from surprise import accuracy
rating_dt = pd.read_csv('ratings.csv')
def Item_basedCF(rating):
    reader = Reader(rating_scale=(0,5))
    data = Dataset.load_from_df(rating,reader)
    my_k = 15
    my_min_k = 5
    my_sim_option = {'name':'pearson','user_based':False}
    algo = KNNWithMeans(k = my_k, min_k = my_min_k, sim_option = my_sim_option)
    trainset, testset = train_test_split(data, test_size=0.25)
    predictions = algo.fit(trainset).test(testset)
    acc = accuracy.rmse(predictions)
    return acc
user_ids = rating_dt["userId"].unique().tolist()
num_all_user = len(user_ids)
np.random.seed(123)
rand_userid = np.random.choice(user_ids, size = int(num_all_user * 0.1), replace=False)
sample_df = rating_dt.loc[rating_dt['userId'].isin(rand_userid)]
sample_df = sample_df[['userId','movieId','rating']]
kNN_model = Item_basedCF(sample_df)
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(sample_df,reader)
my_k = 15
my_min_k = 5
my_sim_option = {'name':'pearson','user_based':False}
algo = KNNWithMeans(k = my_k, min_k = my_min_k, sim_option = my_sim_option)
trainset, testset = train_test_split(data, test_size=0.25)
predictions = algo.fit(trainset).test(testset)
Computing the msd similarity matrix...
Done computing similarity matrix.
np.random.seed(123)
userid_random = np.random.choice(user_ids, 100, replace=False)
rel_list = []
for user in userid_random:
    recommended = []
    rate = []
    for i in predictions:
        if i[0] == user and i[3] >= 4.5:
            rate.append(i[3])
            recommended.append(i[1])
        if len(recommended) == 20:
            break
    indx_sorted = np.flip(np.argsort(rate))
    sorted_recommend = np.array(recommended)[indx_sorted]
    rated_movie = sample_df.loc[sample_df['userId'] == user].movieId.tolist()
    rel = []
    for index, item in enumerate(recommended):
        if item in rated_movie:
            rel.append(1)
        else:
            rel.append(0)
    rel_list.append(rel)
dcg_sum = 0
for r in rel_list:
    for i, value in enumerate(r):
        if value == 1:
            dcg_sum += 1/np.log(i+2)
dcg_sum/100
